# COS360 - Otimização 2020.2
## Bernardo Maiorano e Igor Amaral

$$
\begin{align}
\displaystyle \min \ \  &f(x)\\
\text{sujeito a:}\  \ \ &x \in \Omega\\
\\
f(x_1,x_2,x_3,x_4) &= -30x_1 - 10x_2 - 40x_3 - 12x_4\\
\Omega = \{x \in \mathbb{R}^4 :& \  {33x_1 + 14x_2 + 47x_3 + 11x_4 \le 59\}} \\
\end{align}
$$

##  Estudo da função

In [1]:
import numpy as np
import pandas as pd

In [2]:
coefficients = np.array([-30, -10, -40, -12])
omega = np.array([33, 14, 47, 11])

def f(x):
    return x @ coefficients

def G(x):
    # Restrição de desigualdade
    return omega @ x <= 59

# Gradiente constante
grad_f = coefficients

Temos uma função linear no R4, com apenas uma restrição igualmente linear. Dessa forma, temos uma função convexa, já que a mesma é linear (um plano no R4). Isso garante que, caso haja um mínimo local contido no espaço do domínio, esse mínimo também será global.  Temos também que, pelo fato da função ser linear, teremos um gradiente constante Grad_f = (-30,-10,-40,-12). Além disso, a Hessiana dessa função é uma matriz 4x4 vazia.  Pelo fato desta ser uma função linear, podemos resolver a mesma pelo Simplex. Dessa forma, com a restrição dada e sabendo que no simplex x1, x2, x3 e x4 >= 0, temos o seguinte valor ótimo:

In [3]:
# mínimo pelo simplex: x = [0,0,0,59/11]
f([0,0,0,59/11])

-64.36363636363636

### Penalidade Exterior

Pelo fato desta função ser restrita, para poder utilizar os métodos de programação não-linear, é necessário transformar o problema em um problema irrestrito. Faremos isso a seguir.

Para transformar nosso problema restrito em irrestrito. Temos que usar o método de penalidade externa ou interna.
$ \\ $
Assim, o problema transformado seria: $\min \phi(x,\rho) = f(x) + \rho P(x)$ onde $\rho$ é o parâmetro de penalidade e $P(x)$ é a função de penalidade.

Vamos usar a função de penalidade externa: $\displaystyle P(x) = \sum_{h \in H} h(x)^2 + \sum_{g \in G} \max[0, g(x)]^2$

Para o novo problema temos o gradiente: $\nabla \varPhi(x,\rho) = \nabla f(x) + \rho\nabla P(x) = \begin{cases}
    \nabla f(x) &\text{se}_\ g(x) \le 0\\
    \nabla f(x) + \rho\nabla P(x) &\text{se}_\ g(x) > 0 \\
\end{cases}$

No nosso caso, a função objetivo torna-se: $ \\ $
$$
\phi(x,\rho) = {-30x_1-10x_2-40x_3-12x_4} + \rho \max[0,{33x_1+14x_2+47x_3+11x_4-59}]^2
$$



In [4]:
rho = 10000000

def P_Exterior(x, p = rho):
    return f(x) + p*pow(max(0, omega@x-59),2)

# https://www.wolframalpha.com/input/?i=derivate+%28p*%28max+%280%2C+%2833x%2B14y%2B47z%2B11w-59%29%29%29%5E2
# grad_Pext = grad_f + p*grad(Penalidade)
def grad_Pext(x, p= rho):
    if G(x): return grad_f
    return np.array([(grad_f[i] + (p*2*omega[i]*(x@omega-59))) for i in range(4)])

# https://www.wolframalpha.com/input/?i=hessian+%28%28-30x-10y-40z-12w%29+%2B+%28p*%2833x%2B14y%2B47z%2B11w-59%29%5E2%29%29+with+respect+to+%28x%2Cy%2Cz%2Cw%29
# L4 = 3L1 => det = 0 => matriz não inversivel
def hessian_Pext(x, p= rho):
    hessian = np.zeros((4,4))
    if G(x): return hessian
    for i in range(4):
        hessian[i] = [(p*2*omega[i]*omega[j]) for j in range(4)]
    return hessian

In [5]:
z = np.array([0,0,0,50],  dtype=np.float64)
hessian = hessian_Pext(z, p=1)
print(hessian)
print("\nL4*3 = {} = L1".format(hessian[3]*3))

[[2178.  924. 3102.  726.]
 [ 924.  392. 1316.  308.]
 [3102. 1316. 4418. 1034.]
 [ 726.  308. 1034.  242.]]

L4*3 = [2178.  924. 3102.  726.] = L1


Pelo fato da Hessiana não ser uma matriz inversível, não podemos aplicar o Método de Newton para essa função.

Além disso, para o Método Quase-Newton, teríamos que fazer a diferença dos gradientes, porém para a função dada, conforme explicado anteriormente, o gradiente é constante, portando sua diferença é sempre zero. Isso também nos impede de usar o Método Quase-Newton. Portanto usaremos apenas o Método do Gradiente.

## Implementação

In [6]:
def armijo(x, direction, f, Grad):
    t = 1
    gamma = 0.8
    n = 0.25
    n_iter = 0
    
    while (f(x + t*direction) > f(x) + n*t*Grad(x)@direction):
        t = t*gamma
        n_iter += 1
        
    # print("Armijo:",t,n_iter)
    return t, n_iter

In [7]:
def stop(x, prev_x, n_iter, max_iters = 50):
    return (np.allclose(x, prev_x, atol=1e-16) or n_iter > max_iters)# and G(x)
 
def Gradiente(x, f, Grad):

    n_iter = 0
    prev_x = float('inf')*np.ones(4)
    while not stop(x,prev_x,n_iter):
        direction = -Grad(x)
        t, calls_armijo = armijo(x,direction, f, Grad)
        prev_x = x
        x = x + t*direction
        n_iter += 1
    
    return x, f(x), n_iter, calls_armijo

In [8]:
def quase_Newton(x,f,Grad):

    n_iter = 0
    prev_x = float('inf')*np.ones(4)
    H = np.identity(4,dtype=np.float64)
    while not stop(x,prev_x,n_iter):
        direction = -(H@Grad(x))
        t,calls_armijo = armijo(x,direction, f, Grad)
        prev_x = x
        x = x + t*direction
        n_iter += 1
        # atualizar H (definida positiva)
        p = x - prev_x
        q = Grad(x) - Grad(prev_x)
#         part1 = np.outer(p,p)/(p@q)
#         part2 = np.dot(np.outer(np.dot(H,p),p),H)
#         part3 = p@H@p
#         H = H + part1 -(part2/part3)
        
    return x, f(x), n_iter, calls_armijo

In [9]:
x = np.array([0,0,0,0], dtype=np.float64)
Gradiente(x, P_Exterior, grad_Pext)

(array([0.56333545, 0.18777848, 0.75111393, 0.22533418]),
 -51.526415524699985,
 6,
 86)

In [10]:
x = np.array([0,0,0,0], dtype=np.float64)
quase_Newton(x, P_Exterior, grad_Pext)

(array([0.56333545, 0.18777848, 0.75111393, 0.22533418]),
 -51.526415524699985,
 6,
 86)

In [11]:
n = 10
results = []
while n:
    x0 = np.random.rand(4)
    opt_point, opt_value, n_iter, calls_armijo = Gradiente(x0, P_Exterior, grad_Pext)
    if G(opt_point):
        results.append([x0,n_iter,calls_armijo,opt_point, opt_value])
        n -= 1
        
df = pd.DataFrame(results, columns = ['X0','Iter.','Call.  Armijo','Opt.  Point','Opt.  Value'])
df

,X0,Iter.,Call. Armijo,Opt. Point,Opt. Value
0,"[0.8636258104508732, 0.9205955797254299, 0.550...",7,75,"[0.7599273896183965, 0.8665190209043065, 0.392...",-50.811624
1,"[0.34436343197610275, 0.5926338746503121, 0.08...",5,77,"[0.5784875283523079, 0.6706752401090473, 0.397...",-52.872555
2,"[0.9169385946236072, 0.9153807768061393, 0.710...",7,110,"[0.7519481212590501, 0.8293416558767518, 0.459...",-50.303451
3,"[0.039969832251191106, 0.8591492752982302, 0.6...",6,83,"[0.17333584386762002, 0.9036046125037066, 0.81...",-49.366693
4,"[0.4952413558208262, 0.4736846049324066, 0.573...",5,75,"[0.5522835343117517, 0.49269866442938176, 0.64...",-51.130199
5,"[0.33158012395077496, 0.3802850746842713, 0.33...",6,78,"[0.5111076170845619, 0.4401275723955337, 0.573...",-52.510289
6,"[0.31906633286222863, 0.7879372501544627, 0.33...",6,76,"[0.4727718773756171, 0.8391724316589255, 0.543...",-50.975092
7,"[0.3485909724321393, 0.9759637653411825, 0.213...",7,77,"[0.5597166665137792, 1.046338996701729, 0.4945...",-49.912883
8,"[0.8300429508048457, 0.11410157086166506, 0.10...",6,82,"[1.021406812335683, 0.17788952470527755, 0.358...",-53.256335
9,"[0.5889118959809787, 0.04516907476252274, 0.94...",7,84,"[0.4531343170814991, -0.025636122962656674, 0....",-53.480090


In [12]:
x = np.array([0,0,0,500], dtype=np.float64)
print(G(x))
opt_point, opt_value, n_iter, calls_armijo = Gradiente(x, P_Exterior, grad_Pext)
print(G(opt_point),opt_point,opt_value)

False
True [-47.43989681 -24.73892233 -72.17881914 487.56949843] -1293.0950877044731


In [13]:
x = np.array([-100,-100,-100,5000], dtype=np.float64)
print(G(x))
opt_point, opt_value, n_iter, calls_armijo = Gradiente(x, P_Exterior, grad_Pext)
print(G(opt_point),opt_point,opt_value)

False
True [-497.07034766 -307.0639715  -704.13431915 4895.95713283] -12603.362683130868
